# Create High-k Metal Gate Stack Tutorial

This notebook demonstrates how to create a high-k metal gate stack heterostructure with four materials: Si (substrate), SiO2 (gate oxide), HfO2 (high-k dielectric), and TiN (metal gate).

We'll create a representation of the material from the [QuantumATK tutorial](https://docs.quantumatk.com/tutorials/hkmg_builder/hkmg_builder.html) without the amorphous step.

<img src="https://docs.quantumatk.com/_images/update_square_default_stack.png" alt="High-k Metal Gate Stack" width="700"/>


## 1. Configuration Parameters


In [5]:
from types import SimpleNamespace

from types import SimpleNamespace

# Global parameters
MAX_AREA = 200  # Maximum area for strain matching
MAX_AREA_RATIO_TOL = 0.25  # Maximum area ratio tolerance for strain matching
MAX_ANGLE_TOLERANCE = 0.15  # Maximum angle tolerance for strain matching
MAX_LENGTH_TOLERANCE = 0.15  # Maximum length tolerance for strain matching

# Structure parameters for each layer
STRUCTURE_PARAMS = [
    SimpleNamespace(
        # Silicon substrate
        slab_params=SimpleNamespace(
            miller_indices=(1, 0, 0),
            thickness=4,  # atomic layers
            vacuum=5.0,  # Angstroms
            USE_ORTHOGONAL_C=True
        ),
        termination_formula=None,  # Use default termination for substrate
        interface_distance=None  # No interface for substrate
    ),
    SimpleNamespace(
        # SiO2 layer
        slab_params=SimpleNamespace(
            miller_indices=(1, 0, 0),
            thickness=3,
            vacuum=5.0,
            USE_ORTHOGONAL_C=True
        ),
        termination_formula=None,  # Use default termination for SiO2
        interface_distance=2.5,  # Distance to Si substrate
        vacuum=0.5  # Vacuum for SiO2-terminating interface layer
    ),
    SimpleNamespace(
        # HfO2 layer
        slab_params=SimpleNamespace(
            miller_indices=(0, 0, 1),
            thickness=2,
            vacuum=0.5,
            xy_supercell_matrix=[[2, 0], [0, 2]],
            USE_ORTHOGONAL_C=True
        ),
        termination_formula=None,  # Use default termination for HfO2
        interface_distance=3.0,  # Distance to SiO2
        vacuum=0.5  # Vacuum for HfO2-terminating interface layer
    ),
    SimpleNamespace(
        # TiN layer
        slab_params=SimpleNamespace(
            miller_indices=(0, 0, 1),
            thickness=3,
            vacuum=5.0,  # Add vacuum to final layer
            USE_ORTHOGONAL_C=True,
            xy_supercell_matrix=[[3, 0], [0, 3]]
        ),
        termination_formula=None,  # Use default termination for TiN
        interface_distance=2.5,  # Distance to HfO2
        vacuum=10.0  # Vacuum for TiN-terminating interface layer
    )
]

INTERFACE_1_INDEX = 11 # was found in the previous runs

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [6]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages

    await install_packages("specific_examples")

### 1.3. Get input material
In this notebook we will use materials from Standata.

In [7]:
from mat3ra.made.material import Material
from mat3ra.standata.materials import Materials

materials_1 = [
    Material.create(Materials.get_by_name_first_match("SiO2")),  # SiO2
    Material.create(Materials.get_by_name_first_match("Silicon")),  # Si substrate
]
materials_2 = [
    Material.create(Materials.get_by_name_first_match("HfO2.*MCL")),  # HfO2
    Material.create(Materials.get_by_name_first_match("TiN")),  # TiN
]

In [8]:
from mat3ra.made.tools.build.slab.helpers import create_slab, get_slab_terminations
from mat3ra.made.tools.build.slab.termination_utils import select_slab_termination
from mat3ra.made.tools.build.interface import (
    InterfaceConfiguration,
    ZSLStrainMatchingParameters,
    ZSLStrainMatchingInterfaceBuilder,
    ZSLStrainMatchingInterfaceBuilderParameters
)
from utils.visualize import visualize_materials as visualize

film_1_params = STRUCTURE_PARAMS[1]  # SiO2 layer
substrate_1_params = STRUCTURE_PARAMS[0]  # Silicon substrate

# Get terminations for film (SiO2)
film_1_terminations = get_slab_terminations(
    materials_1[1],  # SiO2
    miller_indices=film_1_params.slab_params.miller_indices
)

# Select termination for film
if film_1_params.termination_formula:
    film_termination = select_slab_termination(film_1_terminations, film_1_params.termination_formula)
else:
    film_termination = film_1_terminations[0]  # Use first available termination

# Get terminations for substrate (Si)
substrate_1_terminations = get_slab_terminations(
    materials_1[0],  # Silicon
    miller_indices=substrate_1_params.slab_params.miller_indices
)

# Select termination for substrate
if substrate_1_params.termination_formula:
    substrate_termination = select_slab_termination(substrate_1_terminations, substrate_1_params.termination_formula)
else:
    substrate_termination = substrate_1_terminations[0]  # Use first available termination

# Create film slab (SiO2)
film_1_slab = create_slab(
    crystal=materials_1[1],  # SiO2
    termination=film_termination,
    miller_indices=film_1_params.slab_params.miller_indices,
    number_of_layers=film_1_params.slab_params.thickness,
    vacuum=film_1_params.slab_params.vacuum,
    use_orthogonal_c=film_1_params.slab_params.USE_ORTHOGONAL_C
)

# Create substrate slab (Si)
substrate_1_slab = create_slab(
    crystal=materials_1[0],  # Silicon
    termination=substrate_termination,
    miller_indices=substrate_1_params.slab_params.miller_indices,
    number_of_layers=substrate_1_params.slab_params.thickness,
    vacuum=substrate_1_params.slab_params.vacuum,
    use_orthogonal_c=substrate_1_params.slab_params.USE_ORTHOGONAL_C
)

# Create interface configuration using the created slabs
interface_config = InterfaceConfiguration(
    film_slab=film_1_slab,
    substrate_slab=substrate_1_slab,
    distance_z=film_1_params.interface_distance,  # Distance to Si substrate
    vacuum=film_1_params.vacuum,  # Vacuum for SiO2-terminating interface layer
)

builder = ZSLStrainMatchingInterfaceBuilder(
    build_parameters=ZSLStrainMatchingInterfaceBuilderParameters(
        strain_matching_parameters=ZSLStrainMatchingParameters(max_area=MAX_AREA, max_area_ratio_tol=MAX_AREA_RATIO_TOL,
                                                               max_angle_tol=MAX_ANGLE_TOLERANCE,
                                                               max_length_tol=MAX_LENGTH_TOLERANCE)
    )
)

interfaces = builder.get_materials(configuration=interface_config)
interface_1 = interfaces[INTERFACE_1_INDEX]


ImportError: cannot import name 'InterfaceConfiguration' from 'mat3ra.made.tools.build.interface' (/Users/mat3ra/code/GREEN/api-examples/.venv-3.11.2/lib/python3.11/site-packages/mat3ra/made/tools/build/interface/__init__.py)

## 2. Create Si/SiO2 Interface
### 2.1. Create Si/SiO2 Interface

In [ ]:
from mat3ra.made.tools.build.slab.helpers import create_slab, get_slab_terminations
from mat3ra.made.tools.build.slab.termination_utils import select_slab_termination
from mat3ra.made.tools.build.interface import (
    InterfaceConfiguration,
    ZSLStrainMatchingParameters,
    ZSLStrainMatchingInterfaceBuilder,
    ZSLStrainMatchingInterfaceBuilderParameters
)
from utils.visualize import visualize_materials as visualize

film_1_params = STRUCTURE_PARAMS[1]  # SiO2 layer
substrate_1_params = STRUCTURE_PARAMS[0]  # Silicon substrate

# Get terminations for film (SiO2)
film_1_terminations = get_slab_terminations(
    materials_1[1],  # SiO2
    miller_indices=film_1_params.slab_params.miller_indices
)

# Select termination for film
if film_1_params.termination_formula:
    film_termination = select_slab_termination(film_1_terminations, film_1_params.termination_formula)
else:
    film_termination = film_1_terminations[0]  # Use first available termination

# Get terminations for substrate (Si)
substrate_1_terminations = get_slab_terminations(
    materials_1[0],  # Silicon
    miller_indices=substrate_1_params.slab_params.miller_indices
)

# Select termination for substrate
if substrate_1_params.termination_formula:
    substrate_termination = select_slab_termination(substrate_1_terminations, substrate_1_params.termination_formula)
else:
    substrate_termination = substrate_1_terminations[0]  # Use first available termination

# Create film slab (SiO2)
film_1_slab = create_slab(
    crystal=materials_1[1],  # SiO2
    termination=film_termination,
    miller_indices=film_1_params.slab_params.miller_indices,
    number_of_layers=film_1_params.slab_params.thickness,
    vacuum=film_1_params.slab_params.vacuum,
    use_orthogonal_c=film_1_params.slab_params.USE_ORTHOGONAL_C
)

# Create substrate slab (Si)
substrate_1_slab = create_slab(
    crystal=materials_1[0],  # Silicon
    termination=substrate_termination,
    miller_indices=substrate_1_params.slab_params.miller_indices,
    number_of_layers=substrate_1_params.slab_params.thickness,
    vacuum=substrate_1_params.slab_params.vacuum,
    use_orthogonal_c=substrate_1_params.slab_params.USE_ORTHOGONAL_C
)

# Create interface configuration using the created slabs
interface_config = InterfaceConfiguration(
    film_slab=film_1_slab,
    substrate_slab=substrate_1_slab,
    distance_z=film_1_params.interface_distance,  # Distance to Si substrate
    vacuum=film_1_params.vacuum,  # Vacuum for SiO2-terminating interface layer
)

builder = ZSLStrainMatchingInterfaceBuilder(
    build_parameters=ZSLStrainMatchingInterfaceBuilderParameters(
        strain_matching_parameters=ZSLStrainMatchingParameters(max_area=MAX_AREA, max_area_ratio_tol=MAX_AREA_RATIO_TOL,
                                                               max_angle_tol=MAX_ANGLE_TOLERANCE,
                                                               max_length_tol=MAX_LENGTH_TOLERANCE)
    )
)

interfaces = builder.get_materials(configuration=interface_config)
interface_1 = interfaces[INTERFACE_1_INDEX]

ImportError: cannot import name 'SlabConfiguration' from 'mat3ra.made.tools.build.slab' (/Users/mat3ra/code/GREEN/api-examples/.venv-3.11.2/lib/python3.11/site-packages/mat3ra/made/tools/build/slab/__init__.py)

### 2.2. Visualize the Si/SiO2 Interface

In [ ]:
visualize(
    interface_1,
    repetitions=[1, 1, 1],
    title="Si/SiO2 Interface"
)

visualize(
    interface_1,
    repetitions=[1, 1, 1],
    title="Si/SiO2 Interface",
    rotation='-90x'
)


## 3. Add HfO2 Layer
### 3.1. Add a layer with SimpleInterfaceBuilder

In [ ]:
from mat3ra.made.tools.build.interface import SimpleInterfaceBuilder, SimpleInterfaceBuilderParameters

film_2_params = STRUCTURE_PARAMS[2]  # HfO2 layer

# Get terminations for HfO2
film_2_terminations = get_slab_terminations(
    materials_2[0],  # HfO2
    miller_indices=film_2_params.slab_params.miller_indices
)

# Select termination for HfO2
if film_2_params.termination_formula:
    film_2_termination = select_slab_termination(film_2_terminations, film_2_params.termination_formula)
else:
    film_2_termination = film_2_terminations[0]  # Use first available termination

# Create HfO2 slab
film_2_slab = create_slab(
    crystal=materials_2[0],  # HfO2
    termination=film_2_termination,
    miller_indices=film_2_params.slab_params.miller_indices,
    number_of_layers=film_2_params.slab_params.thickness,
    vacuum=film_2_params.slab_params.vacuum,
    xy_supercell_matrix=film_2_params.slab_params.xy_supercell_matrix,
    use_orthogonal_c=film_2_params.slab_params.USE_ORTHOGONAL_C
)

# Get terminations for substrate (interface_1)
substrate_2_terminations = get_slab_terminations(
    interface_1,
    miller_indices=(0, 0, 1)
)

# Select first termination for substrate
substrate_2_termination = substrate_2_terminations[0]

print("Film terminations:", film_2_terminations)
print("Substrate terminations:", substrate_2_terminations)

# Create interface configuration
interface_2_config = InterfaceConfiguration(
    film_slab=film_2_slab,
    substrate_slab=interface_1,
    distance_z=film_2_params.interface_distance,  # Distance to SiO2
    vacuum=film_2_params.vacuum  # Vacuum for HfO2-terminating interface layer
)

builder = SimpleInterfaceBuilder(build_parameters=SimpleInterfaceBuilderParameters(scale_film=True, build_slabs=False))
interface_2 = builder.get_material(configuration=interface_2_config)

### 3.2. Visualize the Si/SiO2/HfO2 Interface

In [ ]:
visualize(
    interface_2,
    repetitions=[1, 1, 1],
    title="Si/SiO2/HfO2 Interface"
)

visualize(
    interface_2,
    repetitions=[1, 1, 1],
    title="Si/SiO2/HfO2 Interface",
    rotation='-90x'
)

## 4. Add TiN Layer
### 4.1. Create a TiN Slab

In [9]:
from mat3ra.made.tools.build.supercell import create_supercell

film_3_params = STRUCTURE_PARAMS[3]  # TiN layer

# Get terminations for TiN
film_3_terminations = get_slab_terminations(
    materials_2[1],  # TiN
    miller_indices=film_3_params.slab_params.miller_indices
)

# Select termination for TiN
if film_3_params.termination_formula:
    film_3_termination = select_slab_termination(film_3_terminations, film_3_params.termination_formula)
else:
    film_3_termination = film_3_terminations[0]  # Use first available termination

# Create TiN slab
tin_slab = create_slab(
    crystal=materials_2[1],  # TiN
    termination=film_3_termination,
    miller_indices=film_3_params.slab_params.miller_indices,
    number_of_layers=film_3_params.slab_params.thickness,
    vacuum=film_3_params.slab_params.vacuum,
    xy_supercell_matrix=film_3_params.slab_params.xy_supercell_matrix,
    use_orthogonal_c=film_3_params.slab_params.USE_ORTHOGONAL_C
)

interface_2_flipped = create_supercell(interface_2, scaling_factor=[1, -1, -1])

/Users/mat3ra/code/GREEN/api-examples/.venv-3.11.2/lib/python3.11/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['number']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/Users/mat3ra/code/GREEN/api-examples/.venv-3.11.2/lib/python3.11/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['international']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/Users/mat3ra/code/GREEN/api-examples/.venv-3.11.2/lib/python3.11/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['wyckoffs']) is deprecated.Use attribute interface ({self.__class__.__name__}.{key}) instead
  warnings.warn(
/Users/mat3ra/code/GREEN/api-examples/.venv-3.11.2/lib/python3.11/site-packages/spglib/spglib.py:115: DeprecationWarning: dict interface (SpglibDataset['equivalent_atoms']) is deprecated.Use attribute interface 

NameError: name 'interface_2' is not defined

### 4.2. Get terminations for TiN and Si/SiO2/HfO2

In [ ]:
# Get terminations for TiN slab (already created)
tin_slab_terminations = get_slab_terminations(
    tin_slab,
    miller_indices=(0, 0, 1)
)

# Get terminations for substrate (interface_2_flipped)
substrate_3_terminations = get_slab_terminations(
    interface_2_flipped,
    miller_indices=(0, 0, 1)
)

# Select first terminations
tin_slab_termination = tin_slab_terminations[0]
substrate_3_termination = substrate_3_terminations[0]

print("TiN slab terminations:", tin_slab_terminations)
print("Substrate terminations:", substrate_3_terminations)

### 4.3. Add TiN layer with SimpleInterfaceBuilder

In [ ]:
# Visualize the created slabs from the interface creation
visualize([{"material": film_1_slab, "title": "SiO2 Film"}, {"material": substrate_1_slab, "title": "Si Substrate"}],
          repetitions=[3, 3, 1], rotation="-90x")


In [ ]:
# Create final interface configuration
interface_3_config = InterfaceConfiguration(
    film_slab=tin_slab,
    substrate_slab=interface_2_flipped,
    distance_z=film_3_params.interface_distance,  # Distance to HfO2
    vacuum=film_3_params.vacuum  # Vacuum for TiN-terminating interface layer
)

builder = SimpleInterfaceBuilder(build_parameters=SimpleInterfaceBuilderParameters(scale_film=True, create_slabs=False))

interface_3 = builder.get_material(configuration=interface_3_config)
# invert the interface along the z-axis to match example
interface_3 = create_supercell(interface_3, scaling_factor=[1, -1, -1])

### 4.2. Visualize the Si/SiO2/HfO2/TiN Interface

In [ ]:
visualize(
    interface_3,
    title="Si/SiO2/HfO2/TiN Interface",
    viewer="wave",
    rotation='-90x'
)

## 5. Save final material


In [ ]:
# Visualize HfO2 film slab created earlier
visualize([{"material": film_2_slab, "title": "HfO2 Film"}], repetitions=[3, 3, 1], rotation="-90x")


In [ ]:
# Visualize TiN slab created earlier
visualize([{"material": tin_slab, "title": "TiN Film"}], repetitions=[3, 3, 1], rotation="-90x")


In [ ]:
from utils.jupyterlite import download_content_to_file, set_materials
interface_3.name = "Si-SiO2-HfO2-TiN Heterostructure"
set_materials(interface_3)
download_content_to_file(interface_3, "heterostructure_high_k_metal_gate.json")